## Version Description
In this version, we deploy a Hybrid PINN with adaptive Line Search in _fullBatch_ mode, to predict 8 correction factors in TZ3, viz. for the MTR, the 4 riser ducts and other associated elements

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.io import export_svgs
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [4]:
import re 
def sorted_alphanum(l): 
    """ Sort the given alphanumeric list""" 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

### Data Acquisition

In [5]:
target_df = pd.read_csv('../data/TZ3_dataset.csv')
target_df.head()

,HoV,CAOLH_C38-C40,CAOLH_C40-C42,CAOLH_C42-C44,CAOLH_C44-C46,CAORH_C38-C40,CAORH_C40-C42,CAORH_C42-C44,CAORH_C44-C46,LAOLH_C38-C40,...,R321_HS1,R321_HS2,R330_HS1,R331_HS1,R332_HS1,R333_HS1,R334_HS1,R335_HS1,R336_HS1,R337_HS1
0,A1,0,0,0,0,0,0,0,0,0,...,113,40,0,0,41,41,41,41,41,41
1,A2,0,0,0,0,0,0,0,0,0,...,113,39,0,0,21,21,41,41,41,41
2,A3,0,0,0,0,0,0,0,0,0,...,113,40,0,0,41,41,41,41,41,41
3,A4,0,0,0,0,0,0,0,0,0,...,113,43,0,21,41,41,41,41,41,41
4,A5,0,0,0,0,0,0,0,0,0,...,113,40,21,21,41,41,41,41,41,41


In [6]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 56 columns):
HoV               30 non-null object
CAOLH_C38-C40     30 non-null int64
CAOLH_C40-C42     30 non-null int64
CAOLH_C42-C44     30 non-null int64
CAOLH_C44-C46     30 non-null int64
CAORH_C38-C40     30 non-null int64
CAORH_C40-C42     30 non-null int64
CAORH_C42-C44     30 non-null int64
CAORH_C44-C46     30 non-null int64
LAOLH_C38-C40     30 non-null int64
LAOLH_C40-C42     30 non-null int64
LAOLH_C42-C44     30 non-null int64
LAOLH_C44-C46     30 non-null int64
LAORH_C38-C40     30 non-null int64
LAORH_C40-C42     30 non-null int64
LAORH_C42-C44     30 non-null int64
LAORH_C44-C46     30 non-null int64
MIXP              30 non-null float64
MIXT              30 non-null float64
AMBP              30 non-null float64
AMBT              30 non-null float64
Density(kg/m³)    30 non-null float64
CAOLH_SumFlow     30 non-null float64
CAORH_SumFlow     30 non-null float64
LAOLH_SumFlow     

In [7]:
# Define Restrictors and their parameters
MTR = 'R300_HD'
CLRs =  ['R310_HS1', 'R311_HS1', 'R312_HS1', 'R313_HS1']
CAORs = ['R320_HS1', 'R320_HS2', 'R321_HS1', 'R321_HS2']
MHRS = CLRs + CAORs

# Duct Areas
MHR_Duct_Areas = {'R310_HS1':0.0129658,'R311_HS1':0.01327322896,'R312_HS1':0.01767145868,'R313_HS1':0.01767145868,
                  'R320_HS1':0.0108506,'R320_HS2':0.00470159,'R321_HS1':0.0108506,'R321_HS2':0.00470158}

restrictor_duct_areas = {'R300_HD':0.02405281875, 'riser_LAO_RHS':0.00516612, 'riser_LAO_LHS':0.0052845, 'riser_CAO_RHS':0.0173304, 'riser_CAO_LHS':0.0173301 }

#3D Correction Factors for MultiHole Restrictors
MTR_cf = 0.812
MHR_cf = {'R310_HS1':0.832, 'R311_HS1':0.879, 'R312_HS1':0.727, 'R313_HS1':0.771, 
          'R320_HS1':0.912, 'R320_HS2':0.974, 'R321_HS1':0.817, 'R321_HS2':1.047}


## Define Initial Zeta values for riser ducts from CFD

riser_duct_zetasCFD = { 'riser_LAO_RHS':0.730110153034089, 'riser_LAO_LHS':1.91790022798789, 'riser_CAO_RHS':11.4767551741128, 'riser_CAO_LHS':11.8379787151916}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [8]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [9]:
_,target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,restrictor_duct_areas['R300_HD'],MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R300_HD_Zeta3D']].head()

,R300_HD,R300_HD_Zeta3D
0,116,5.076081
1,114,5.642979
2,124,3.307892
3,124,3.307892
4,122,3.685326


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [10]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [11]:
# Calculate Zeta for Multi-Hole Restrictors
hole_dia = 8 # MHR Hole Dia
for clr in CLRs:
    MHR_nr_holes = target_df[clr].values
    _, target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele, hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORs:
    CAOR_nr_holes = target_df[caor].values
    _,target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

**Store Loss Coefficient Values and Names for all Restrictors**:

In [12]:
# Gather all column names required for the input loss coefficients
from TZ3_FDDN_Lib import zeta_values, name_variables

In [13]:
print('FDDN Elements:\n',name_variables)

FDDN Elements:
 ['TZ-3_zmix', 'MTR-300', 'riser_LAO_RHS', 'riser_LAO_LHS', 'riser_CAO_RHS', 'riser_CAO_LHS', 'R-310', 'R-311', 'R-312', 'R-313', 'LAOR-330', 'LAOR-332', 'LAOR-334', 'LAOR-336', 'LAOR-331', 'LAOR-333', 'LAOR-335', 'LAOR-337', 'CAO-320_fwd', 'CAO-320_mid', 'CAO-320_aft', 'CAO-321_fwd', 'CAO-321_mid', 'CAO-321_aft', 'LAO-330', 'LAO-332', 'LAO-334', 'LAO-336', 'LAO-331', 'LAO-333', 'LAO-335', 'LAO-337', 'CAOR-320_fwd', 'CAOR-320_mid', 'CAOR-320_aft', 'CAOR-321_fwd', 'CAOR-321_mid', 'CAOR-321_aft', 'pp2310pdloss', 'pp2311pdloss', 'pp2312pdloss', 'pp2313pdloss', '23120-320_F', '23120-320_M', '23120-320_A', '23130-321_F', '23130-321_M', '23130-321_A', '3302.2-3303', '3303-3304', '3302.2-3305', '3305-3306', '2301-3302.1', '2301-3302.2']


In [14]:
cf_riser_initial = 1
idx = target_df.columns.get_loc("R300_HD_Zeta3D")
target_df.insert(loc=(idx+1), column='riser_LAO_RHS_Zeta3D', value= float(zeta_values[2]) * cf_riser_initial)
target_df.insert(loc=(idx+2), column='riser_LAO_LHS_Zeta3D', value= float(zeta_values[3]) * cf_riser_initial)
target_df.insert(loc=(idx+3), column='riser_CAO_RHS_Zeta3D', value= float(zeta_values[4]) * cf_riser_initial)
target_df.insert(loc=(idx+4), column='riser_CAO_LHS_Zeta3D', value= float(zeta_values[5]) * cf_riser_initial)
target_df['riser_3302_2_3305_Zeta3D']= float(zeta_values[50]) * cf_riser_initial
target_df['riser_3302_1_Zeta3D']= float(zeta_values[52]) * cf_riser_initial
target_df['riser_3302_2_Zeta3D']= float(zeta_values[53]) * cf_riser_initial

In [15]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
fm_flag_col_names = sorted_alphanum([col for col in target_df.columns if 'FM' in col])
fm_flag_col_names_idx = [target_df.columns.get_loc(col) for col in fm_flag_col_names if col in target_df]
fp2_flag_col_names = sorted_alphanum([col for col in target_df.columns if 'FP2' in col])
fp2_flag_col_names_idx = [target_df.columns.get_loc(col) for col in fp2_flag_col_names if col in target_df]
fddn_input_col_names = fm_flag_col_names + fp2_flag_col_names + zeta_col_names
print(fddn_input_col_names)

['LAO_330_FM', 'LAO_331_FM', 'LAO_332_FM', 'LAO_333_FM', 'LAO_334_FM', 'LAO_335_FM', 'LAO_330_FP2', 'LAO_331_FP2', 'LAO_332_FP2', 'LAO_333_FP2', 'LAO_334_FP2', 'LAO_335_FP2', 'R300_HD_Zeta3D', 'riser_LAO_RHS_Zeta3D', 'riser_LAO_LHS_Zeta3D', 'riser_CAO_RHS_Zeta3D', 'riser_CAO_LHS_Zeta3D', 'R310_HS1_Zeta3D', 'R311_HS1_Zeta3D', 'R312_HS1_Zeta3D', 'R313_HS1_Zeta3D', 'R320_HS1_Zeta3D', 'R320_HS2_Zeta3D', 'R321_HS1_Zeta3D', 'R321_HS2_Zeta3D', 'riser_3302_2_3305_Zeta3D', 'riser_3302_1_Zeta3D', 'riser_3302_2_Zeta3D']


In [16]:
target_df[zeta_col_names]

,R300_HD_Zeta3D,riser_LAO_RHS_Zeta3D,riser_LAO_LHS_Zeta3D,riser_CAO_RHS_Zeta3D,riser_CAO_LHS_Zeta3D,R310_HS1_Zeta3D,R311_HS1_Zeta3D,R312_HS1_Zeta3D,R313_HS1_Zeta3D,R320_HS1_Zeta3D,R320_HS2_Zeta3D,R321_HS1_Zeta3D,R321_HS2_Zeta3D,riser_3302_2_3305_Zeta3D,riser_3302_1_Zeta3D,riser_3302_2_Zeta3D
0,5.076081,0.73011,1.9179,11.476755,11.837979,28.753818,54.552445,7.864128,8.909238,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
1,5.642979,0.73011,1.9179,11.476755,11.837979,28.753818,82.805829,6.089576,6.733670,4.124839,9.755073,2.885978,7.658574,1.880442,1.291464,2.508788
2,3.307892,0.73011,1.9179,11.476755,11.837979,25.516955,47.357346,8.982575,9.966643,3.221556,7.124641,2.885978,7.097864,1.880442,1.291464,2.508788
3,3.307892,0.73011,1.9179,11.476755,11.837979,35.466354,33.489790,9.397860,9.966643,3.221556,5.281530,2.885978,5.677335,1.880442,1.291464,2.508788
4,3.685326,0.73011,1.9179,11.476755,11.837979,16.449171,32.156533,8.400799,9.966643,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
5,3.685326,0.73011,1.9179,11.476755,11.837979,16.449171,39.622740,10.544011,11.999959,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
6,4.102714,0.73011,1.9179,11.476755,11.837979,28.753818,54.552445,7.864128,8.909238,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
7,5.076081,0.73011,1.9179,11.476755,11.837979,35.466354,63.319432,4.402668,5.263191,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
8,5.076081,0.73011,1.9179,11.476755,11.837979,35.466354,63.319432,4.402668,5.263191,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788
9,4.102714,0.73011,1.9179,11.476755,11.837979,22.740948,63.319432,6.217884,6.876629,3.221556,7.694577,2.885978,7.097864,1.880442,1.291464,2.508788


### Training data Preparation

In [17]:
from TZ3_8CF import NormbyMax, PINN

In [18]:
features2scale = ['R300_HD_Zeta3D','riser_3302_1_Zeta3D','riser_3302_2_Zeta3D','riser_3302_2_3305_Zeta3D','riser_LAO_RHS_Zeta3D','riser_LAO_LHS_Zeta3D','riser_CAO_RHS_Zeta3D','riser_CAO_LHS_Zeta3D','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow','TZ3_Flow']
features_max, df_rescaled = NormbyMax(target_df,features2scale)
print('Max value of all features:',features_max)

Max value of all features: {'R300_HD_Zeta3D': 5.642978728418095, 'riser_3302_1_Zeta3D': 1.29146402905416, 'riser_3302_2_Zeta3D': 2.50878846999567, 'riser_3302_2_3305_Zeta3D': 1.88044178924511, 'riser_LAO_RHS_Zeta3D': 0.730110153034089, 'riser_LAO_LHS_Zeta3D': 1.91790022798789, 'riser_CAO_RHS_Zeta3D': 11.4767551741128, 'riser_CAO_LHS_Zeta3D': 11.8379787151916, 'LAORH_SumFlow': 104.206563, 'LAOLH_SumFlow': 99.792513, 'CAORH_SumFlow': 160.021532, 'CAOLH_SumFlow': 159.438776, 'TZ3_Flow': 505.45200199999994}


In [19]:
features_max_df = pd.DataFrame(data = features_max, index=[0])
print('Max value of all input features for normalization')
features_max_df

Max value of all input features for normalization


,R300_HD_Zeta3D,riser_3302_1_Zeta3D,riser_3302_2_Zeta3D,riser_3302_2_3305_Zeta3D,riser_LAO_RHS_Zeta3D,riser_LAO_LHS_Zeta3D,riser_CAO_RHS_Zeta3D,riser_CAO_LHS_Zeta3D,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow,TZ3_Flow
0,5.642979,1.291464,2.508788,1.880442,0.73011,1.9179,11.476755,11.837979,104.206563,99.792513,160.021532,159.438776,505.452002


In [20]:
V_max_org = features_max_df[['TZ3_Flow','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow']]
V_max_org

,TZ3_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,505.452002,104.206563,99.792513,160.021532,159.438776


In [21]:
# Manually selected Indices of train-val-test data split
train_data_idx =  [16, 0, 4, 5] # [16, 0, 4, 5, 6, 13, 28, 14,  18, 1, 9, 12, 2, 20, 7, 3, 24, 27]
val_data_idx = [17, 19, 21] # [17, 19, 21, 25, 33, 27, 8, 11,  22]
test_data_idx = [10, 15, 23, 26, 29 ]

In [22]:
# Store indices and values of train data
input_features = ['R300_HD_Zeta3D']
train_data = df_rescaled[input_features].iloc[train_data_idx]
val_data = df_rescaled[input_features].iloc[val_data_idx]
test_data = df_rescaled[input_features].iloc[test_data_idx]
print("Number of training samples:", len(train_data), "\nNumber of validation samples:", len(val_data), "\nNumber of test samples:", len(test_data))

# Train data rparams
train_R300HD_series   = target_df['R300_HD'].iloc[train_data_idx]
train_riser_LAO_RHS_Zeta3D_series = target_df['riser_LAO_RHS_Zeta3D'].iloc[train_data_idx]
train_riser_LAO_LHS_Zeta3D_series = target_df['riser_LAO_LHS_Zeta3D'].iloc[train_data_idx]
train_riser_CAO_RHS_Zeta3D_series = target_df['riser_CAO_RHS_Zeta3D'].iloc[train_data_idx]
train_riser_CAO_LHS_Zeta3D_series = target_df['riser_CAO_LHS_Zeta3D'].iloc[train_data_idx]
train_riser_3302_2_3305_Zeta3D_series = target_df['riser_3302_2_3305_Zeta3D'].iloc[train_data_idx]
train_riser_3302_1_Zeta3D_series = target_df['riser_3302_1_Zeta3D'].iloc[train_data_idx]
train_riser_3302_2_Zeta3D_series = target_df['riser_3302_2_Zeta3D'].iloc[train_data_idx]
rparams_series_train = (train_R300HD_series,train_riser_LAO_RHS_Zeta3D_series,train_riser_LAO_LHS_Zeta3D_series,train_riser_CAO_RHS_Zeta3D_series,train_riser_CAO_LHS_Zeta3D_series,train_riser_3302_2_3305_Zeta3D_series,train_riser_3302_1_Zeta3D_series,train_riser_3302_2_Zeta3D_series)

# Validation data rparams
val_R300HD_series   = target_df['R300_HD'].iloc[val_data_idx]
val_riser_LAO_RHS_Zeta3D_series = target_df['riser_LAO_RHS_Zeta3D'].iloc[val_data_idx]
val_riser_LAO_LHS_Zeta3D_series = target_df['riser_LAO_LHS_Zeta3D'].iloc[val_data_idx]
val_riser_CAO_RHS_Zeta3D_series = target_df['riser_CAO_RHS_Zeta3D'].iloc[val_data_idx]
val_riser_CAO_LHS_Zeta3D_series = target_df['riser_CAO_LHS_Zeta3D'].iloc[val_data_idx]
val_riser_3302_2_3305_Zeta3D_series = target_df['riser_3302_2_3305_Zeta3D'].iloc[val_data_idx]
val_riser_3302_1_Zeta3D_series = target_df['riser_3302_1_Zeta3D'].iloc[val_data_idx]
val_riser_3302_2_Zeta3D_series = target_df['riser_3302_2_Zeta3D'].iloc[val_data_idx]
rparams_series_val = (val_R300HD_series,val_riser_LAO_RHS_Zeta3D_series,val_riser_LAO_LHS_Zeta3D_series,val_riser_CAO_RHS_Zeta3D_series,val_riser_CAO_LHS_Zeta3D_series,val_riser_3302_2_3305_Zeta3D_series,val_riser_3302_1_Zeta3D_series,val_riser_3302_2_Zeta3D_series)

Number of training samples: 4 
Number of validation samples: 3 
Number of test samples: 5


### Predict $c_f$ with Physics Informed Neural Network

In [23]:
### Set the hyperparameters here ###
nr_iterations = 25
learning_rates = [0, 0.0001] ### IMPORTANT PARAMETER
train_batch_size = len(train_data) 
val_batch_size = len(val_data)

N_i = train_data.shape[1] # No. of feature columns in the dataframe
hl_dim = [2,8] ### IMPORTANT PARAMETER
output_nodes = 8
network_dim = (N_i,hl_dim,output_nodes)

epsi_factor = 0.1 # 0.01
MTR_epsi = np.float(MTR_cf * epsi_factor)
riser_LAO_RHS_epsi = np.float(1 * epsi_factor)
riser_LAO_LHS_epsi = np.float(1 * epsi_factor)
riser_CAO_RHS_epsi = np.float(1 * epsi_factor)
riser_CAO_LHS_epsi = np.float(1 * epsi_factor)
riser_3302_2_3305_epsi = np.float(1 * epsi_factor)
riser_3302_1_epsi = np.float(1 * epsi_factor)
riser_3302_2_epsi = np.float(1 * epsi_factor)
epsi_tuple = (MTR_epsi, riser_LAO_RHS_epsi, riser_LAO_LHS_epsi, riser_CAO_RHS_epsi, riser_CAO_LHS_epsi, riser_3302_2_3305_epsi, riser_3302_1_epsi, riser_3302_2_epsi)

In [24]:
print('MTR Epsi:',MTR_epsi, '\triser_LAO_RHS_epsi:',riser_LAO_RHS_epsi, '\triser_LAO_LHS_epsi:',riser_LAO_LHS_epsi, '\triser_CAO_RHS_epsi:',riser_CAO_RHS_epsi, '\triser_CAO_LHS_epsi:',riser_CAO_LHS_epsi, '\triser_3302_1_epsi:',riser_3302_1_epsi, '\triser_3302_2_epsi:',riser_3302_2_epsi, '\triser_3302_2_3305_epsi:',riser_3302_2_3305_epsi)

MTR Epsi: 0.08120000000000001 	riser_LAO_RHS_epsi: 0.1 	riser_LAO_LHS_epsi: 0.1 	riser_CAO_RHS_epsi: 0.1 	riser_CAO_LHS_epsi: 0.1 	riser_3302_1_epsi: 0.1 	riser_3302_2_epsi: 0.1 	riser_3302_2_3305_epsi: 0.1


#### Train data points sequentially - Single Data Point Training Mode

In [25]:
%%time
network = PINN(epsi_tuple, activation_function_choice = 'lrelu', error_loss_choice = 'full') #cf_only, full, simplified, hyperbola_regularization                                                                                         
trained_model, MSE_flowloss_hist_train, error_loss_hist_train, cf_hist_train, MSE_flowloss_hist_val, error_loss_hist_val, cf_hist_val, flow_delta_val_mean = network.train_batch_LS(learning_rates, network_dim, input_features,train_data, rparams_series_train, train_data_idx, train_batch_size, restrictor_duct_areas, target_df, fddn_input_col_names, V_max_org, val_batch_size, val_data, rparams_series_val, val_data_idx, epochs=nr_iterations) 

Activation Function selected: lrelu
Loss Function & Delta-k Used: full

USING ENTIRE TRAIN DATASET in current TRAINING Batch: [16, 0, 4, 5]
SEED value: 13
Nr of Hidden layers: 2 ;	Nr_Neurons in last hidden layer: 8
Initializing weights from first hidden layer to last hidden layer
FIRST - INITIAL WEIGHTS & BIASES:
W1: [array([-0.712, 0.754])] <class 'numpy.ndarray'> Shape: (1, 2)
b1: [1.0, 1.0] <class 'numpy.ndarray'> Shape: (2,)
W2: [array([-0.147, -0.266, 0.452, 1.45, -0.656, -0.548, -0.309, 0.387]), array([-0.604, 0.329, -0.51, 0.398, -0.223, -0.304, 0.576, -0.838])] <class 'numpy.ndarray'> Shape: (2, 8)
b2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (8,)
W3: [array([-0.0157, 0.16, 0.476, 0.188, 0.477, 0.304, 0.523, -0.37]), array([-0.279, -0.446, 0.199, -0.086, 0.323, 0.112, 0.045, 0.76]), array([0.214, -0.00947, -0.348, 0.421, 0.337, -0.384, -0.0513, 0.0841]), array([-0.58, -0.0983, 0.495, -0.571, 0.174, 0.669, -0.22, -0.16]), array([0.0769, 0.182, 0.14

Flowrate_delta (FDDN - LTR): [array([4.08, -1.34, -2.6, 6.29, 1.72]), array([0.884, -12.2, 1.87, 7.01, 4.24]), array([15.3, 1.8, 2.89, 6.2, 4.39]), array([-4.91, -7.43, 0.0387, 2.32, 0.166])] <class 'numpy.ndarray'> (4, 5)
f2: -792.9279445594727 ;	lr2: 0.0005233149350537833 ;	MSE_FlowError: 33.78122062238615 ;	Error: 3.671038424980467
f2 < 0 -> Use Brent
zfunc - fullBatch length: 4

MTR-Cf: 0.8084707132791569 ; riser_LAO_RHS-Cf: 0.9864854162533163 ; riser_LAO_LHS-Cf: 1.0157258620336662 ; riser_CAO_RHS-Cf: 0.9845617631494648 ; riser_CAO_LHS-Cf: 0.9919740821022408 ; riser_3302_2_3305-Cf: 0.9975740307436487 ; riser_3302_1-Cf: 0.9908391747291133 ; riser_3302_2-Cf: 0.9311668148979797
Row ID: 16 	HoV: ['F2']

MTR-Cf: 0.8311460262215848 ; riser_LAO_RHS-Cf: 0.9719742527643307 ; riser_LAO_LHS-Cf: 1.01874798050432 ; riser_CAO_RHS-Cf: 0.9657973013950261 ; riser_CAO_LHS-Cf: 0.9883395713357609 ; riser_3302_2_3305-Cf: 1.0261138223359736 ; riser_3302_1-Cf: 1.0045300101727157 ; riser_3302_2-Cf: 0.9920

Flowrate_delta (FDDN - LTR): [array([-1.39, -2.12, -3.26, 4.11, -0.124]), array([-4.37, -12.9, 1.16, 4.9, 2.45]), array([9.89, 0.911, 2.1, 4.2, 2.67]), array([-10, -8.31, -0.685, 0.431, -1.48])] <class 'numpy.ndarray'> (4, 5)
f1: 106.49184839971964 ;	lr1: 0.0001 ;	MSE_FlowError: 27.531842143330692 ;	Error: 3.4781930044352265

Learning Rate 2: 0.002971738516024433

MTR-Cf: 0.7163936558910886 ; riser_LAO_RHS-Cf: 0.9450416600584435 ; riser_LAO_LHS-Cf: 0.9926018444196372 ; riser_CAO_RHS-Cf: 1.2085259213574455 ; riser_CAO_LHS-Cf: 1.0320545829610217 ; riser_3302_2_3305-Cf: 0.8213486653735245 ; riser_3302_1-Cf: 1.0004551447410384 ; riser_3302_2-Cf: 0.8804938512123934
Row ID: 16 	HoV: ['F2']

MTR-Cf: 0.7395241479885071 ; riser_LAO_RHS-Cf: 0.9306712156371801 ; riser_LAO_LHS-Cf: 0.9950356573358468 ; riser_CAO_RHS-Cf: 1.191000603554642 ; riser_CAO_LHS-Cf: 1.0282771583069346 ; riser_3302_2_3305-Cf: 0.848412400271848 ; riser_3302_1-Cf: 1.014300885621385 ; riser_3302_2-Cf: 0.9409275710706804
Row ID:

Flowrate_delta (FDDN - LTR): [array([5.29, 1.36, -1.63, 2.09, 3.46]), array([1.8, -10.4, 2.94, 3.26, 6.01]), array([17.2, 5.28, 3.93, 2.36, 5.66]), array([-2.9, -3.97, 0.976, -1.23, 1.32])] <class 'numpy.ndarray'> (4, 5)
f0: 341.1371934073077 ;	lr0: 0 ;	MSE_FlowError: 31.129576566306618 ;	Error: 2.89739500836797

MTR-Cf: 0.737231201102833 ; riser_LAO_RHS-Cf: 0.947645346042493 ; riser_LAO_LHS-Cf: 0.9929531992810982 ; riser_CAO_RHS-Cf: 1.198398869581124 ; riser_CAO_LHS-Cf: 1.0342532206944264 ; riser_3302_2_3305-Cf: 0.8289608600420497 ; riser_3302_1-Cf: 1.0009103233171306 ; riser_3302_2-Cf: 0.8879723857175967
Row ID: 16 	HoV: ['F2']

MTR-Cf: 0.7604062286310846 ; riser_LAO_RHS-Cf: 0.9332775554677502 ; riser_LAO_LHS-Cf: 0.9953683634724787 ; riser_CAO_RHS-Cf: 1.18085217839211 ; riser_CAO_LHS-Cf: 1.0304548807850953 ; riser_3302_2_3305-Cf: 0.856040448095152 ; riser_3302_1-Cf: 1.0147690716446554 ; riser_3302_2-Cf: 0.9484921700048095
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.6969473230009029 ; riser_LAO


MTR-Cf: 0.7301284721440828 ; riser_LAO_RHS-Cf: 0.9766761216354478 ; riser_LAO_LHS-Cf: 1.0014989367339968 ; riser_CAO_RHS-Cf: 1.2410604647568328 ; riser_CAO_LHS-Cf: 1.0682907947653584 ; riser_3302_2_3305-Cf: 0.7842074959405483 ; riser_3302_1-Cf: 0.9949233239114161 ; riser_3302_2-Cf: 0.7816394253430898
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - LTR): [array([-0.507, 0.599, -2.34, 0.0922, 1.14]), array([-3.78, -11.1, 2.17, 1.3, 3.8]), array([11.5, 4.44, 3.09, 0.487, 3.52]), array([-8.34, -4.81, 0.195, -3.01, -0.716])] <class 'numpy.ndarray'> (4, 5)
f1: 11.60560402211882 ;	lr1: 0.0001 ;	MSE_FlowError: 22.10442456484595 ;	Error: 2.542164468233926

Learning Rate 2: 0.0021290540598289313

MTR-Cf: 0.7787562847313847 ; riser_LAO_RHS-Cf: 0.9348424482443427 ; riser_LAO_LHS-Cf: 1.0038859910885893 ; riser_CAO_RHS-Cf: 1.200307355134134 ; riser_CAO_LHS-Cf: 1.113217449354119 ; riser_3302_2_3305-Cf: 0.8022461777831871 ; riser_3302_1-Cf: 1.0057272054270128 ; riser_3302_2-Cf: 0.8836231401599235
Row I

...


MTR-Cf: 0.7819133517840735 ; riser_LAO_RHS-Cf: 0.9161976039563051 ; riser_LAO_LHS-Cf: 1.1150589652104286 ; riser_CAO_RHS-Cf: 1.2621346362871482 ; riser_CAO_LHS-Cf: 1.1912988610625241 ; riser_3302_2_3305-Cf: 0.8471408872073827 ; riser_3302_1-Cf: 1.0415367356252596 ; riser_3302_2-Cf: 0.8690517574818514
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.71935300617492 ; riser_LAO_RHS-Cf: 0.9642217112747489 ; riser_LAO_LHS-Cf: 1.1343462990616018 ; riser_CAO_RHS-Cf: 1.3194624405512139 ; riser_CAO_LHS-Cf: 1.226034643265653 ; riser_3302_2_3305-Cf: 0.7890395751875037 ; riser_3302_1-Cf: 1.0321035990055614 ; riser_3302_2-Cf: 0.6861249633348862
Row ID: 4 	HoV: ['A5']

MTR-Cf: 0.71935300617492 ; riser_LAO_RHS-Cf: 0.9642217112747489 ; riser_LAO_LHS-Cf: 1.1343462990616018 ; riser_CAO_RHS-Cf: 1.3194624405512139 ; riser_CAO_LHS-Cf: 1.226034643265653 ; riser_3302_2_3305-Cf: 0.7890395751875037 ; riser_3302_1-Cf: 1.0321035990055614 ; riser_3302_2-Cf: 0.6861249633348862
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - LTR):


MTR-Cf: 0.7821655296010852 ; riser_LAO_RHS-Cf: 0.9168795525613447 ; riser_LAO_LHS-Cf: 1.1176131666935742 ; riser_CAO_RHS-Cf: 1.2645103084217046 ; riser_CAO_LHS-Cf: 1.1932141328829236 ; riser_3302_2_3305-Cf: 0.8504044625062224 ; riser_3302_1-Cf: 1.0421679038313476 ; riser_3302_2-Cf: 0.8687040130959107
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.7196242692201154 ; riser_LAO_RHS-Cf: 0.9650462409916101 ; riser_LAO_LHS-Cf: 1.1372614275952304 ; riser_CAO_RHS-Cf: 1.3220032114817062 ; riser_CAO_LHS-Cf: 1.2282902497669443 ; riser_3302_2_3305-Cf: 0.7925031313975439 ; riser_3302_1-Cf: 1.033147953760854 ; riser_3302_2-Cf: 0.685508233202883
Row ID: 4 	HoV: ['A5']

MTR-Cf: 0.7196242692201154 ; riser_LAO_RHS-Cf: 0.9650462409916101 ; riser_LAO_LHS-Cf: 1.1372614275952304 ; riser_CAO_RHS-Cf: 1.3220032114817062 ; riser_CAO_LHS-Cf: 1.2282902497669443 ; riser_3302_2_3305-Cf: 0.7925031313975439 ; riser_3302_1-Cf: 1.033147953760854 ; riser_3302_2-Cf: 0.685508233202883
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - LTR


MTR-Cf: 0.7786705839149721 ; riser_LAO_RHS-Cf: 0.9171092597280601 ; riser_LAO_LHS-Cf: 1.12002034291649 ; riser_CAO_RHS-Cf: 1.2653720723959923 ; riser_CAO_LHS-Cf: 1.1944510265433697 ; riser_3302_2_3305-Cf: 0.8533180578006021 ; riser_3302_1-Cf: 1.0424054633554467 ; riser_3302_2-Cf: 0.8671086545478998
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.7161973093524661 ; riser_LAO_RHS-Cf: 0.9653726728118464 ; riser_LAO_LHS-Cf: 1.1398457246754725 ; riser_CAO_RHS-Cf: 1.3229948770206166 ; riser_CAO_LHS-Cf: 1.2296947451269902 ; riser_3302_2_3305-Cf: 0.7954769484022195 ; riser_3302_1-Cf: 1.0336456362040578 ; riser_3302_2-Cf: 0.6837985854829302
Row ID: 4 	HoV: ['A5']

MTR-Cf: 0.7161973093524661 ; riser_LAO_RHS-Cf: 0.9653726728118464 ; riser_LAO_LHS-Cf: 1.1398457246754725 ; riser_CAO_RHS-Cf: 1.3229948770206166 ; riser_CAO_LHS-Cf: 1.2296947451269902 ; riser_3302_2_3305-Cf: 0.7954769484022195 ; riser_3302_1-Cf: 1.0336456362040578 ; riser_3302_2-Cf: 0.6837985854829302
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - L


MTR-Cf: 0.7820282312708307 ; riser_LAO_RHS-Cf: 0.917934015675665 ; riser_LAO_LHS-Cf: 1.1214530108555028 ; riser_CAO_RHS-Cf: 1.2679773034835309 ; riser_CAO_LHS-Cf: 1.1960508628935704 ; riser_3302_2_3305-Cf: 0.8553902672269027 ; riser_3302_1-Cf: 1.042953251197607 ; riser_3302_2-Cf: 0.867941653774265
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.7195238035230067 ; riser_LAO_RHS-Cf: 0.9663101182738039 ; riser_LAO_LHS-Cf: 1.1416215245819124 ; riser_CAO_RHS-Cf: 1.3257154568712686 ; riser_CAO_LHS-Cf: 1.2316161574621907 ; riser_3302_2_3305-Cf: 0.7977706155608608 ; riser_3302_1-Cf: 1.0345366486189074 ; riser_3302_2-Cf: 0.6843628940318902
Row ID: 4 	HoV: ['A5']

MTR-Cf: 0.7195238035230067 ; riser_LAO_RHS-Cf: 0.9663101182738039 ; riser_LAO_LHS-Cf: 1.1416215245819124 ; riser_CAO_RHS-Cf: 1.3257154568712686 ; riser_CAO_LHS-Cf: 1.2316161574621907 ; riser_3302_2_3305-Cf: 0.7977706155608608 ; riser_3302_1-Cf: 1.0345366486189074 ; riser_3302_2-Cf: 0.6843628940318902
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - LT


MTR-Cf: 0.7775567823592684 ; riser_LAO_RHS-Cf: 0.9185854361579824 ; riser_LAO_LHS-Cf: 1.1251580001570258 ; riser_CAO_RHS-Cf: 1.2698774798073331 ; riser_CAO_LHS-Cf: 1.1983150636054103 ; riser_3302_2_3305-Cf: 0.859588527453472 ; riser_3302_1-Cf: 1.0433537851776677 ; riser_3302_2-Cf: 0.8655219832870629
Row ID: 0 	HoV: ['A1']

MTR-Cf: 0.7151426268017524 ; riser_LAO_RHS-Cf: 0.96712622412862 ; riser_LAO_LHS-Cf: 1.1456667738238684 ; riser_CAO_RHS-Cf: 1.3278264297219395 ; riser_CAO_LHS-Cf: 1.234200542422391 ; riser_3302_2_3305-Cf: 0.8021129430427183 ; riser_3302_1-Cf: 1.0353951789923646 ; riser_3302_2-Cf: 0.6817200761694329
Row ID: 4 	HoV: ['A5']

MTR-Cf: 0.7151426268017524 ; riser_LAO_RHS-Cf: 0.96712622412862 ; riser_LAO_LHS-Cf: 1.1456667738238684 ; riser_CAO_RHS-Cf: 1.3278264297219395 ; riser_CAO_LHS-Cf: 1.234200542422391 ; riser_3302_2_3305-Cf: 0.8021129430427183 ; riser_3302_1-Cf: 1.0353951789923646 ; riser_3302_2-Cf: 0.6817200761694329
Row ID: 5 	HoV: ['A6']
Flowrate_delta (FDDN - LTR): 

In [26]:
print("MSE_flowloss_train_history:",MSE_flowloss_hist_train)

MSE_flowloss_train_history: [28.032132840176054, 31.129576566306618, 22.154265025129355, 21.528346487401414, 21.149208068582205, 21.187901201632187, 21.004048314277338, 21.320273619621297, 20.823335354822234, 20.913368839894652, 20.80978904549328, 20.79215241638852, 20.721351073084072, 20.73143923893806, 20.66438477836173, 20.64341772654762, 20.66810386747476, 20.410261119196402, 20.449220340361087, 20.370258708515856, 20.38238080891896, 20.307172101069717, 20.367781657402915, 20.267849262502416, 20.268432013670427]


In [27]:
print("MSE_flowloss_val_history:",MSE_flowloss_hist_val)

MSE_flowloss_val_history: [37.715894689209385, 28.753442050554735, 27.74706540168713, 28.006192290227048, 25.990349111970822, 26.841949917326357, 25.587786717197996, 26.962174457707906, 24.92824486679298, 25.4750246963157, 24.53675062131305, 25.05578854083667, 24.309745720395593, 24.797089242920954, 24.058479280648733, 24.565911283137325, 24.539514857137323, 23.830821455247012, 24.486850735902994, 23.768341902794635, 24.310681153822944, 23.56436398951321, 24.245904530231133, 23.583335894074587, 24.24554331832304]


In [28]:
print("Error_loss_train_history:",error_loss_hist_train)

Error_loss_train_history: [3.549801209261632, 2.89739500836797, 2.550582285637824, 2.424565849510664, 2.413540604195197, 2.3953202968982725, 2.392125222388712, 2.3802042386508595, 2.3673156324517084, 2.3642620189525987, 2.3629111533640486, 2.3560592297406955, 2.3544167773623457, 2.349561560816083, 2.3471158007684845, 2.340734337719271, 2.3427258932838084, 2.3229173055920156, 2.319704150298145, 2.320315159622982, 2.3169420396357627, 2.316970198866891, 2.315618628156502, 2.3141676298736704, 2.3101819251497417]


In [29]:
print("Error_loss_val_history:",error_loss_hist_val)

Error_loss_val_history: [7.492377559151471, 5.83694880382619, 5.739590486859853, 5.457766182388271, 5.355938635405496, 5.30305360707034, 5.234207883161046, 5.170870918821088, 5.0822719446216285, 5.071761576566573, 5.0187376500872265, 5.026109597228118, 4.953551156162754, 4.959579530686768, 4.89233509900674, 4.904341456288098, 4.901443771742037, 4.760803503451225, 4.775055742205185, 4.748288265045729, 4.762514430386618, 4.7297838812322235, 4.753937935714138, 4.722958166239223, 4.745738530339956]


In [30]:
print('Flow Delta Val Mean:',flow_delta_val_mean)

Flow Delta Val Mean: [4.586996987220535, 4.09750860527487, 3.740223552970244, 3.8091165319701537, 3.675703983814008, 3.7147536966154195, 3.6320671357872687, 3.716647544386319, 3.569730126074339, 3.598525852437203, 3.538038191936248, 3.56895435513113, 3.513991622328036, 3.5438225714011415, 3.488887489046192, 3.5198017731632634, 3.518710356484843, 3.455426176082548, 3.5015811597821975, 3.4510684996144807, 3.490607191800272, 3.4356686852719274, 3.487355909033577, 3.4357459326686004, 3.48854743568954]


In [31]:
cf_df_train = pd.DataFrame(data=cf_hist_train)
cf_df_train

,HoV,MTR_cf_hat,riser_LAO_RHS_cf_hat,riser_LAO_LHS_cf_hat,riser_CAO_RHS_cf_hat,riser_CAO_LHS_cf_hat,riser_3302_2_3305_cf_hat,riser_3302_1_cf_hat,riser_3302_2_cf_hat,Epoch
0,F2,0.757911,0.934569,1.126464,1.290762,1.205325,0.835558,1.032339,0.804742,24
1,A1,0.780606,0.919374,1.126840,1.272592,1.200084,0.861764,1.043978,0.866187,24
2,A5,0.718166,0.968038,1.147715,1.330669,1.236314,0.804521,1.036393,0.682101,24
3,A6,0.718166,0.968038,1.147715,1.330669,1.236314,0.804521,1.036393,0.682101,24


In [32]:
cf_df_val = pd.DataFrame(data=cf_hist_val)
cf_df_val

,HoV,MTR_cf_hat_val,riser_LAO_RHS_cf_hat_val,riser_LAO_LHS_cf_hat_val,riser_CAO_RHS_cf_hat_val,riser_CAO_LHS_cf_hat_val,riser_3302_2_3305_cf_hat_val,riser_3302_1_cf_hat_val,riser_3302_2_cf_hat_val,Epoch
0,F3,0.718166,0.968038,1.147715,1.330669,1.236314,0.804521,1.036393,0.682101,24
1,J1,0.737033,0.952150,1.137627,1.311725,1.221603,0.819254,1.034469,0.740319,24
2,P1,0.706130,0.975252,1.149526,1.341073,1.222729,0.780614,1.044470,0.642344,24


### Plot Training Losses

In [33]:
hover = [('Epoch', '@x'), ('Value','@y')]

**MSE FlowError Loss**

In [34]:
p1 = figure(title = 'Full batch MSE FlowLoss performance',width=1600, tooltips=hover)
p1.line(list(range(1, len(MSE_flowloss_hist_train)+1)), MSE_flowloss_hist_train, line_width=2, color='blue', alpha=0.8, legend = 'Train Data')
p1.line(list(range(1, len(MSE_flowloss_hist_val)+1)), MSE_flowloss_hist_val, line_width=2, color='green', alpha=0.8, legend = 'Validation Data')
p1.yaxis.axis_label = "MSE (FDDN_V, LTR_V)"
p1.xaxis.axis_label = "Epochs"
p1.legend.location = "top_right"
p1.legend.click_policy="hide"
show(p1)

**Error Loss**

In [35]:
p2 = figure(title = 'FullBatch Train and Validation Loss Results', width=1250, tooltips=hover)
p2.line(list(range(1,len(error_loss_hist_train)+1)), error_loss_hist_train, line_width= 2, color='green', alpha=0.8, legend='Train Loss')  
p2.circle(list(range(1,len(error_loss_hist_train)+1)), error_loss_hist_train, line_width= 5, color= 'green', alpha=0.75, size = 4,  legend='Train Loss')
p2.line(list(range(1,len(error_loss_hist_val)+1)), error_loss_hist_val, line_width= 2, line_dash = 'dashed', color='blue', alpha=0.8, legend='Validation Loss')  
p2.circle(list(range(1,len(error_loss_hist_val)+1)), error_loss_hist_val, line_width= 5, color= 'blue', alpha=0.45, size = 4, legend='Validation Loss')
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
p2.xaxis.axis_label_text_font_size = '12pt'
p2.yaxis.axis_label_text_font_size = '12pt'
p2.xaxis.major_label_text_font_size= '11pt'
p2.yaxis.major_label_text_font_size= '11pt'
p2.legend.label_text_font_size = '12pt'
p2.xaxis.major_label_standoff = 15
p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ3_8CF_fullbatch_Train_Val_Loss_Results.svg")
show(p2)

In [36]:
p3 = figure(title = 'Flow_delta_Mean Results', width=1250, tooltips=hover)
x = list(range(1,len(flow_delta_val_mean)+1))
p3.line(x, flow_delta_val_mean, line_width=2, color='tomato', alpha=0.8)
p3.circle(x, flow_delta_val_mean, line_width=5, color= 'tomato', alpha=0.9, size = 2)
p3.yaxis.axis_label = "Flow_delta_Mean"
p3.xaxis.axis_label = "Epochs"
p3.legend.click_policy="hide"
p3.xaxis.axis_label_text_font_size = '12pt'
p3.yaxis.axis_label_text_font_size = '12pt'
p3.xaxis.major_label_text_font_size= '11pt'
p3.yaxis.major_label_text_font_size= '11pt'
p3.legend.label_text_font_size = '12pt'
p3.xaxis.major_label_standoff = 15
p3.output_backend = 'svg'
# export_svgs(p3, filename="plots/TZ3_8CF_fullbatch_FlowDeltaMeanResults.svg")
show(p3)

### Make Predictions on Test Data Points

In [37]:
# Test data rparams
test_R300HD_series   = target_df['R300_HD'].iloc[test_data_idx]
test_riser_LAO_RHS_Zeta3D_series = target_df['riser_LAO_RHS_Zeta3D'].iloc[test_data_idx]
test_riser_LAO_LHS_Zeta3D_series = target_df['riser_LAO_LHS_Zeta3D'].iloc[test_data_idx]
test_riser_CAO_RHS_Zeta3D_series = target_df['riser_CAO_RHS_Zeta3D'].iloc[test_data_idx]
test_riser_CAO_LHS_Zeta3D_series = target_df['riser_CAO_LHS_Zeta3D'].iloc[test_data_idx]
test_riser_3302_2_3305_Zeta3D_series = target_df['riser_3302_2_3305_Zeta3D'].iloc[test_data_idx]
test_riser_3302_1_Zeta3D_series = target_df['riser_3302_1_Zeta3D'].iloc[test_data_idx]
test_riser_3302_2_Zeta3D_series = target_df['riser_3302_2_Zeta3D'].iloc[test_data_idx]
rparams_series_test = (test_R300HD_series,test_riser_LAO_RHS_Zeta3D_series,test_riser_LAO_LHS_Zeta3D_series,test_riser_CAO_RHS_Zeta3D_series,test_riser_CAO_LHS_Zeta3D_series,test_riser_3302_2_3305_Zeta3D_series,test_riser_3302_1_Zeta3D_series,test_riser_3302_2_Zeta3D_series)

In [38]:
test_data_HoVs = target_df['HoV'].iloc[test_data_idx].values
V_true_LTR_testdata, V_hat_FDDN_testdata, delta_V_testdata = [], [], []
MTR_cf_arr, riser_LAO_RHS_cf_arr ,riser_LAO_LHS_cf_arr , riser_CAO_RHS_cf_arr , riser_CAO_LHS_cf_arr, riser_3302_2_3305_cf_arr, riser_3302_1_cf_arr, riser_3302_2_cf_arr  = [], [], [], [], [], [], [], []

for row_id, MTR_dia, riser_LAO_RHS_Zeta, riser_LAO_LHS_Zeta, riser_CAO_RHS_Zeta, riser_CAO_LHS_Zeta, riser_3302_2_3305_Zeta, riser_3302_1_Zeta, riser_3302_2_Zeta in zip(test_data_idx, test_R300HD_series, test_riser_LAO_RHS_Zeta3D_series, test_riser_LAO_LHS_Zeta3D_series, test_riser_CAO_RHS_Zeta3D_series, test_riser_CAO_LHS_Zeta3D_series, test_riser_3302_2_3305_Zeta3D_series, test_riser_3302_1_Zeta3D_series, test_riser_3302_2_Zeta3D_series):
    rparam = (MTR_dia, riser_LAO_RHS_Zeta, riser_LAO_LHS_Zeta, riser_CAO_RHS_Zeta, riser_CAO_LHS_Zeta, riser_3302_2_3305_Zeta, riser_3302_1_Zeta, riser_3302_2_Zeta)
    cfs_test_tuple, V_LTR, V_hat_FDDN, delta_V = network.predict(trained_model, test_data.loc[row_id].values, row_id, rparam, restrictor_duct_areas, fddn_input_col_names, target_df)  
    MTR_cf_test, riser_LAO_RHS_cf_test, riser_LAO_LHS_cf_test, riser_CAO_RHS_cf_test, riser_CAO_LHS_cf_test, riser_3302_2_3305_cf_test, riser_3302_1_cf_test, riser_3302_2_cf_test = cfs_test_tuple
    V_true_LTR_testdata.append(V_LTR)
    V_hat_FDDN_testdata.append(V_hat_FDDN)
    delta_V_testdata.append(delta_V)
    MTR_cf_arr.append(MTR_cf_test)
    riser_LAO_RHS_cf_arr.append(riser_LAO_RHS_cf_test)
    riser_LAO_LHS_cf_arr.append(riser_LAO_LHS_cf_test)
    riser_CAO_RHS_cf_arr.append(riser_CAO_RHS_cf_test)
    riser_CAO_LHS_cf_arr.append(riser_CAO_LHS_cf_test)
    riser_3302_2_3305_cf_arr.append(riser_3302_2_3305_cf_test)
    riser_3302_1_cf_arr.append(riser_3302_1_cf_test)
    riser_3302_2_cf_arr.append(riser_3302_2_cf_test)    

testdata_cf_predictions_df = pd.DataFrame(data = {'HoV':test_data_HoVs, 'MTR_cf_hat':MTR_cf_arr, 'riser_LAO_RHS_cf_hat':riser_LAO_RHS_cf_arr, 'riser_LAO_LHS_cf_hat':riser_LAO_LHS_cf_arr, 'riser_CAO_RHS_cf_hat':riser_CAO_RHS_cf_arr, 'riser_CAO_LHS_cf_hat':riser_CAO_LHS_cf_arr,
                                                 'riser_3302_2_3305_cf_hat':riser_3302_2_3305_cf_arr,'riser_3302_1_cf_hat':riser_3302_1_cf_arr,'riser_3302_2_cf_hat':riser_3302_2_cf_arr})


MTR-Cf: 0.6975178953874441 ; riser_LAO_RHS-Cf: 0.9785354394341335 ; riser_LAO_LHS-Cf: 1.1478502839195435 ; riser_CAO_RHS-Cf: 1.3474356496472546 ; riser_CAO_LHS-Cf: 1.1982464889632003 ; riser_3302_2_3305-Cf: 0.7541846399666536 ; riser_3302_1-Cf: 1.0546505220208762 ; riser_3302_2-Cf: 0.6122160426262724
Row ID: 10 	HoV: ['C5']
Predicted CFs: (0.6975178953874441, 0.9785354394341335, 1.1478502839195435, 1.3474356496472546, 1.1982464889632003, 0.7541846399666536, 1.0546505220208762, 0.6122160426262724)

MTR-Cf: 0.7579107524997188 ; riser_LAO_RHS-Cf: 0.9345685758334701 ; riser_LAO_LHS-Cf: 1.1264638544147 ; riser_CAO_RHS-Cf: 1.2907618716254805 ; riser_CAO_LHS-Cf: 1.2053249676620565 ; riser_3302_2_3305-Cf: 0.8355584549122341 ; riser_3302_1-Cf: 1.0323390444235574 ; riser_3302_2-Cf: 0.8047424654140725
Row ID: 15 	HoV: ['F1']
Predicted CFs: (0.7579107524997188, 0.9345685758334701, 1.1264638544147, 1.2907618716254805, 1.2053249676620565, 0.8355584549122341, 1.0323390444235574, 0.8047424654140725)


In [39]:
delta_V_df = pd.concat(delta_V_testdata,ignore_index=True)
V_true_LTR_test_df = pd.concat(V_true_LTR_testdata,ignore_index=True)
V_hat_FDDN_test_df = pd.concat(V_hat_FDDN_testdata,ignore_index=True)

In [40]:
print("LTR_Test_Data_Predictions",'\t\t\t\t\t\tFDDN_Test_data_predictions')
display_side_by_side(V_true_LTR_test_df, V_hat_FDDN_test_df)

LTR_Test_Data_Predictions 						FDDN_Test_data_predictions


,TZ3_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,496.390638,103.520012,88.147681,152.449484,152.273461
1,450.328899,74.549359,73.978195,150.292008,151.509337
2,461.905662,76.488007,81.180295,153.180513,151.056847
3,505.452002,104.206563,99.792513,150.972237,150.480689
4,446.640374,73.677229,75.968614,148.790059,148.204472
#,TZ3_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,493.135830,98.813157,88.198295,153.004131,153.120246
1,450.302270,76.743846,71.492281,151.147412,150.918730
2,458.550067,62.519870,85.059169,155.770144,155.200884
3,496.314406,98.071882,95.751213,151.252021,151.239290


In [41]:
delta_V_df

,Delta_TZ3_Flow,Delta_LAORH_SumFlow,Delta_LAOLH_SumFlow,Delta_CAORH_SumFlow,Delta_CAOLH_SumFlow
0,-3.254808,-4.706855,0.050614,0.554647,0.846785
1,-0.026630,2.194487,-2.485915,0.855404,-0.590606
2,-3.355595,-13.968137,3.878874,2.589631,4.144037
3,-9.137596,-6.134681,-4.041300,0.279784,0.758601
4,0.304989,-9.727334,4.518316,2.694401,2.819607


In [42]:
testdata_cf_predictions_df

,HoV,MTR_cf_hat,riser_LAO_RHS_cf_hat,riser_LAO_LHS_cf_hat,riser_CAO_RHS_cf_hat,riser_CAO_LHS_cf_hat,riser_3302_2_3305_cf_hat,riser_3302_1_cf_hat,riser_3302_2_cf_hat
0,C5,0.697518,0.978535,1.147850,1.347436,1.198246,0.754185,1.054651,0.612216
1,F1,0.757911,0.934569,1.126464,1.290762,1.205325,0.835558,1.032339,0.804742
2,S1,0.718166,0.968038,1.147715,1.330669,1.236314,0.804521,1.036393,0.682101
3,S4,0.718166,0.968038,1.147715,1.330669,1.236314,0.804521,1.036393,0.682101
4,T2,0.757911,0.934569,1.126464,1.290762,1.205325,0.835558,1.032339,0.804742
